In [1]:
import pandas as pd

In [2]:
# read the files
no_DNF_results_data_file = pd.read_csv(r"/home/winter-storm/f1-data-project/erdos_ds_f1/cleaned_data/f1db-races-race-results-NO-DNF.csv")
races_data_file = pd.read_csv(r"/home/winter-storm/f1-data-project/erdos_ds_f1/data_f1db/f1db-races-race-results.csv")

# Create list of unique race ids
race_id_list = no_DNF_results_data_file['raceId'].unique()

/tmp/ipykernel_954256/3409359182.py:3: DtypeWarning: Columns (31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  races_data_file = pd.read_csv(r"/home/winter-storm/f1-data-project/erdos_ds_f1/data_f1db/f1db-races-race-results.csv")


In [3]:
# Get average local position change for each race id
# Write a function to generate ALS for an input of race_id list

def get_correct_positions(position):
    position_copy = position.copy()
    n = len(position)

    for y in range(n):
        if y+1 in position:
            position_copy_index = position_copy.index(y+1)
            position_copy.pop(position_copy_index)
        if y+1 not in position:
            position_copy.sort()
            for z in range(len(position_copy)):
                position_index = position.index(position_copy[z])
                position[position_index] = y+1
                y += 1
            break

    return position

def generate_ALPC_data(input_race_id_list):
    avg_local_pos = []

    for race_id in input_race_id_list:
        race_data = no_DNF_results_data_file[
            (no_DNF_results_data_file['raceId'] == race_id) & 
            (no_DNF_results_data_file['positionDisplayOrder'] > 0)
        ]

        # Get final positions
        final_positions = race_data['positionDisplayOrder'].tolist()
        n = len(final_positions)

        if n < 2:
            avg_local_pos.append(0.0)
            continue

        true_final_positions = get_correct_positions(final_positions)

        this_sum = 0
        for i in range(n):
            summand = 0
            for j in range(n):
                if i == j: continue
                local_change = (i - j) - (true_final_positions.index(i+1) - true_final_positions.index(j+1))
                summand += local_change
            this_sum += abs(summand)

        # Normalize by number of pairs
        #this_sum *= 2  # because we only looped over i < j
        avg = this_sum / (n * (n - 1))
        avg_local_pos.append({'raceId': race_id, 'ALPC': avg})

    return pd.DataFrame(avg_local_pos)
generate_ALPC_data(race_id_list).to_csv('/home/winter-storm/f1-data-project/erdos_ds_f1/Patrick/Feature Data Files/ALPC.csv',
                                        index=False)